In [113]:
import string
import re

from pickle import dump
from unicodedata import normalize
from numpy import array
from pickle import load
from numpy.random import rand
from numpy.random import shuffle
from pickle import load
from numpy import array



In [114]:
#import tensorflow as tf

In [115]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [116]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu


In [117]:
def read_file(file_path):
    file = open(file_path,mode='rt',encoding='utf-8')
    data = file.read()
    file.close()
    return data

In [118]:
data = read_file('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\deu.txt')

In [119]:
len(data)

32439301

In [120]:
data[0:200]

'Go.\tGeh.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)\nHi.\tHallo!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)\nHi.\tGrüß Gott!\tCC-BY 2.0'

In [121]:
data_list = data.split('\n')

In [122]:
len(data_list)

208487

In [123]:
data_list[0:10]

['Go.\tGeh.\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8597805 (Roujin)',
 'Hi.\tHallo!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #380701 (cburgmer)',
 'Hi.\tGrüß Gott!\tCC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #659813 (Esperantostern)',
 'Run!\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #941078 (Fingerhut)',
 'Run.\tLauf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #4008918 (JSakuragi) & #941078 (Fingerhut)',
 'Wow!\tPotzdonner!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122382 (Pfirsichbaeumchen)',
 'Wow!\tDonnerwetter!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #2122391 (Pfirsichbaeumchen)',
 'Fire!\tFeuer!\tCC-BY 2.0 (France) Attribution: tatoeba.org #1829639 (Spamster) & #1958697 (Tamy)',
 'Help!\tHilfe!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #575889 (MUIRIEL)',
 'Help!\tZu Hülf!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084

In [124]:
pairs = [line.split('\t') for line in  data_list]

In [125]:
pairs[10:20]

[['Stop!',
  'Stopp!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #448320 (FeuDRenais) & #626467 (jakov)'],
 ['Wait!',
  'Warte!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #1744314 (belgavox) & #2122378 (Pfirsichbaeumchen)'],
 ['Wait.',
  'Warte.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #3048304 (camilozeta) & #8597806 (Roujin)'],
 ['Begin.',
  'Fang an.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #6102432 (mailohilohi) & #4942826 (Hans_Adler)'],
 ['Go on.',
  'Mach weiter.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2230774 (CK) & #6625701 (Felixjp)'],
 ['Hello!',
  'Hallo!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #380701 (cburgmer)'],
 ['Hurry!',
  'Beeil dich!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #1182334 (Hideousss) & #131 (MUIRIEL)'],
 ['Hurry!',
  'Schnell!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #1182334 (Hideousss) & #1169837 (Haehnchenpaella)'],
 ['I ran.',
  'Ich rannte.',
  'CC-BY 2.0 (France) Attributi

In [126]:
line=['I ran.',
  'Ich rannte.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #5828607 (CK) & #6643977 (Felixjp)']

In [127]:
[word.translate(str.maketrans('', '', string.punctuation)) for word in line]

['I ran',
 'Ich rannte',
 'CCBY 20 France Attribution tatoebaorg 5828607 CK  6643977 Felixjp']

In [128]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [129]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [130]:
"I hello".isalpha()

False

In [131]:
def clean_pair_data(lines):
    cleaned = list()
    # preparing regex for char filter
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize words on white space
            line = line.split()
            # get only lowercase
            line = [word.lower() for word in line]
            # remove punctuations from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return cleaned

In [132]:
clean_pair = clean_pair_data(pairs)

In [133]:
clean_pair[0:20]

[['go', 'geh', 'ccby france attribution tatoebaorg cm roujin'],
 ['hi', 'hallo', 'ccby france attribution tatoebaorg cm cburgmer'],
 ['hi', 'gru gott', 'ccby france attribution tatoebaorg cm esperantostern'],
 ['run', 'lauf', 'ccby france attribution tatoebaorg papabear fingerhut'],
 ['run', 'lauf', 'ccby france attribution tatoebaorg jsakuragi fingerhut'],
 ['wow',
  'potzdonner',
  'ccby france attribution tatoebaorg zifre pfirsichbaeumchen'],
 ['wow',
  'donnerwetter',
  'ccby france attribution tatoebaorg zifre pfirsichbaeumchen'],
 ['fire', 'feuer', 'ccby france attribution tatoebaorg spamster tamy'],
 ['help', 'hilfe', 'ccby france attribution tatoebaorg lukaszpp muiriel'],
 ['help',
  'zu hulf',
  'ccby france attribution tatoebaorg lukaszpp pfirsichbaeumchen'],
 ['stop', 'stopp', 'ccby france attribution tatoebaorg feudrenais jakov'],
 ['wait',
  'warte',
  'ccby france attribution tatoebaorg belgavox pfirsichbaeumchen'],
 ['wait', 'warte', 'ccby france attribution tatoebaorg c

In [134]:
dump(clean_pair,open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\cleaned-english-to-german.pkl','wb'))

In [135]:
data = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\cleaned-english-to-german.pkl','rb'))

In [136]:
data[0:20]

[['go', 'geh', 'ccby france attribution tatoebaorg cm roujin'],
 ['hi', 'hallo', 'ccby france attribution tatoebaorg cm cburgmer'],
 ['hi', 'gru gott', 'ccby france attribution tatoebaorg cm esperantostern'],
 ['run', 'lauf', 'ccby france attribution tatoebaorg papabear fingerhut'],
 ['run', 'lauf', 'ccby france attribution tatoebaorg jsakuragi fingerhut'],
 ['wow',
  'potzdonner',
  'ccby france attribution tatoebaorg zifre pfirsichbaeumchen'],
 ['wow',
  'donnerwetter',
  'ccby france attribution tatoebaorg zifre pfirsichbaeumchen'],
 ['fire', 'feuer', 'ccby france attribution tatoebaorg spamster tamy'],
 ['help', 'hilfe', 'ccby france attribution tatoebaorg lukaszpp muiriel'],
 ['help',
  'zu hulf',
  'ccby france attribution tatoebaorg lukaszpp pfirsichbaeumchen'],
 ['stop', 'stopp', 'ccby france attribution tatoebaorg feudrenais jakov'],
 ['wait',
  'warte',
  'ccby france attribution tatoebaorg belgavox pfirsichbaeumchen'],
 ['wait', 'warte', 'ccby france attribution tatoebaorg c

In [137]:
len(data)

208487

In [138]:
# Train Test SPlit :
initial_dataset = data[:10000]
shuffle(initial_dataset)
train_data = initial_dataset[:9000]
test_data = initial_dataset[9000:]

In [139]:
train_data

[['youre useless',
  'du bist nutzlos',
  'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
 ['who cheated',
  'wer hat geschummelt',
  'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
 ['he was thirsty',
  'er war durstig',
  'ccby france attribution tatoebaorg pliiganto manfredo'],
 ['ok i agree',
  'okay ich bin einverstanden',
  'ccby france attribution tatoebaorg ck vortarulo'],
 ['beat it',
  'zisch ab',
  'ccby france attribution tatoebaorg cm pfirsichbaeumchen'],
 ['keep it', 'behaltet es', 'ccby france attribution tatoebaorg ck raggione'],
 ['mary is pretty',
  'maria ist hubsch',
  'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
 ['we looked up',
  'wir schauten nach oben',
  'ccby france attribution tatoebaorg ck raggione'],
 ['did you call',
  'hast du gerufen',
  'ccby france attribution tatoebaorg ck espi'],
 ['hold still',
  'halten sie still',
  'ccby france attribution tatoebaorg ck yorwba'],
 ['show it to her',
  'zeig ihn ihr',
  'ccb

In [140]:
initial_dataset

[['youre useless',
  'du bist nutzlos',
  'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
 ['who cheated',
  'wer hat geschummelt',
  'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
 ['he was thirsty',
  'er war durstig',
  'ccby france attribution tatoebaorg pliiganto manfredo'],
 ['ok i agree',
  'okay ich bin einverstanden',
  'ccby france attribution tatoebaorg ck vortarulo'],
 ['beat it',
  'zisch ab',
  'ccby france attribution tatoebaorg cm pfirsichbaeumchen'],
 ['keep it', 'behaltet es', 'ccby france attribution tatoebaorg ck raggione'],
 ['mary is pretty',
  'maria ist hubsch',
  'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
 ['we looked up',
  'wir schauten nach oben',
  'ccby france attribution tatoebaorg ck raggione'],
 ['did you call',
  'hast du gerufen',
  'ccby france attribution tatoebaorg ck espi'],
 ['hold still',
  'halten sie still',
  'ccby france attribution tatoebaorg ck yorwba'],
 ['show it to her',
  'zeig ihn ihr',
  'ccb

In [141]:
np_initial_dataset = array(initial_dataset)
np_initial_dataset[:,0]

array(['youre useless', 'who cheated', 'he was thirsty', ...,
       'can you skate', 'i sympathize', 'i hate spinach'], dtype='<U69')

In [142]:
#initial_dataset = data[:10000]
shuffle(np_initial_dataset)
train_data = np_initial_dataset[:9000]
test_data = np_initial_dataset[9000:]

In [162]:
train_data

array([['tom cooks well', 'tom kann gut kochen',
        'ccby france attribution tatoebaorg ck pfirsichbaeumchen'],
       ['keep it up', 'nicht nachlassen',
        'ccby france attribution tatoebaorg ck tamy'],
       ['come here tom', 'komm her tom',
        'ccby france attribution tatoebaorg ck wolfgangth'],
       ...,
       ['i live here', 'ich lebe hier',
        'ccby france attribution tatoebaorg ck esperantostern'],
       ['take your time', 'lasst euch zeit',
        'ccby france attribution tatoebaorg ck yorwba'],
       ['are you done', 'seid ihr fertig',
        'ccby france attribution tatoebaorg ck zaghawa']], dtype='<U69')

In [143]:
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))
 
# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [144]:
train_data[:,0]

array(['tom cooks well', 'keep it up', 'come here tom', ...,
       'i live here', 'take your time', 'are you done'], dtype='<U69')

In [145]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [146]:
def max_sentence_length(lines):
    return max(len(line.split()) for line in lines)

In [147]:
english_tokenizer = create_tokenizer(np_initial_dataset[:,0])
german_tokenizer = create_tokenizer(np_initial_dataset[:,1])


In [148]:
english_tokenizer.index_word

{1: 'tom',
 2: 'i',
 3: 'it',
 4: 'is',
 5: 'you',
 6: 'a',
 7: 'im',
 8: 'me',
 9: 'was',
 10: 'its',
 11: 'we',
 12: 'he',
 13: 'go',
 14: 'do',
 15: 'this',
 16: 'to',
 17: 'youre',
 18: 'are',
 19: 'that',
 20: 'were',
 21: 'ill',
 22: 'my',
 23: 'can',
 24: 'dont',
 25: 'get',
 26: 'come',
 27: 'she',
 28: 'toms',
 29: 'the',
 30: 'they',
 31: 'who',
 32: 'be',
 33: 'here',
 34: 'have',
 35: 'up',
 36: 'in',
 37: 'like',
 38: 'well',
 39: 'not',
 40: 'thats',
 41: 'him',
 42: 'love',
 43: 'on',
 44: 'take',
 45: 'am',
 46: 'us',
 47: 'keep',
 48: 'need',
 49: 'home',
 50: 'let',
 51: 'stop',
 52: 'did',
 53: 'no',
 54: 'know',
 55: 'want',
 56: 'help',
 57: 'got',
 58: 'away',
 59: 'hes',
 60: 'has',
 61: 'cant',
 62: 'try',
 63: 'may',
 64: 'how',
 65: 'lost',
 66: 'now',
 67: 'one',
 68: 'look',
 69: 'there',
 70: 'out',
 71: 'will',
 72: 'what',
 73: 'saw',
 74: 'busy',
 75: 'lets',
 76: 'too',
 77: 'see',
 78: 'theyre',
 79: 'your',
 80: 'down',
 81: 'just',
 82: 'for',
 83: '

In [149]:
a='A'
b='B'

In [150]:
#print("1.{},2.{}".format(a))

In [151]:
def save_object(content, filename):
    dump(content, open(filename, 'wb'))
    print('Saved: %s' % filename)

In [152]:
save_object(english_tokenizer,"C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\english_tokenizer.pkl")
save_object(german_tokenizer,"C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\german_tokenizer.pkl")

Saved: C:\Users\kvoth\Desktop\Datasets\deu-eng\english_tokenizer.pkl
Saved: C:\Users\kvoth\Desktop\Datasets\deu-eng\german_tokenizer.pkl


In [104]:
english_tokenizer = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\english_tokenizer.pkl','rb'))
english_tokenizer.index_word

{1: 'tom',
 2: 'i',
 3: 'it',
 4: 'is',
 5: 'you',
 6: 'a',
 7: 'im',
 8: 'me',
 9: 'was',
 10: 'its',
 11: 'we',
 12: 'he',
 13: 'go',
 14: 'do',
 15: 'this',
 16: 'to',
 17: 'youre',
 18: 'are',
 19: 'that',
 20: 'were',
 21: 'ill',
 22: 'can',
 23: 'my',
 24: 'dont',
 25: 'get',
 26: 'come',
 27: 'she',
 28: 'toms',
 29: 'the',
 30: 'they',
 31: 'who',
 32: 'be',
 33: 'here',
 34: 'have',
 35: 'in',
 36: 'up',
 37: 'like',
 38: 'well',
 39: 'not',
 40: 'him',
 41: 'thats',
 42: 'love',
 43: 'on',
 44: 'take',
 45: 'am',
 46: 'us',
 47: 'keep',
 48: 'need',
 49: 'home',
 50: 'let',
 51: 'stop',
 52: 'did',
 53: 'no',
 54: 'know',
 55: 'want',
 56: 'help',
 57: 'away',
 58: 'got',
 59: 'cant',
 60: 'hes',
 61: 'has',
 62: 'try',
 63: 'how',
 64: 'lost',
 65: 'may',
 66: 'now',
 67: 'one',
 68: 'look',
 69: 'there',
 70: 'out',
 71: 'saw',
 72: 'what',
 73: 'will',
 74: 'busy',
 75: 'lets',
 76: 'see',
 77: 'too',
 78: 'theyre',
 79: 'your',
 80: 'just',
 81: 'down',
 82: 'wait',
 83: 

In [51]:
type(english_tokenizer)

keras_preprocessing.text.Tokenizer

In [52]:
#[386 18 500 0 0] encode for :  dogs are smart
english_tokenizer.index_word.get(500)

'smart'

In [53]:
english_vocab_size = len(english_tokenizer.word_index)+1
max_english_length = max_sentence_length(np_initial_dataset[:,0])
print("English Vocab Size : ",english_vocab_size)
print("Longest English Sentence Length : ", max_english_length)


English Vocab Size :  2214
Longest English Sentence Length :  5


In [54]:
german_vocab_size = len(german_tokenizer.word_index)+1
max_german_length = max_sentence_length(np_initial_dataset[:,1])
print("German Vocab Size : ",german_vocab_size)
print("Longest German Sentence Length : ", max_german_length)


German Vocab Size :  3526
Longest German Sentence Length :  9


In [55]:
english_tokenizer.word_index

{'tom': 1,
 'i': 2,
 'it': 3,
 'is': 4,
 'you': 5,
 'a': 6,
 'im': 7,
 'me': 8,
 'was': 9,
 'its': 10,
 'we': 11,
 'he': 12,
 'go': 13,
 'do': 14,
 'this': 15,
 'to': 16,
 'youre': 17,
 'are': 18,
 'that': 19,
 'were': 20,
 'ill': 21,
 'can': 22,
 'my': 23,
 'dont': 24,
 'get': 25,
 'come': 26,
 'she': 27,
 'toms': 28,
 'the': 29,
 'they': 30,
 'who': 31,
 'be': 32,
 'here': 33,
 'have': 34,
 'in': 35,
 'up': 36,
 'like': 37,
 'well': 38,
 'not': 39,
 'him': 40,
 'thats': 41,
 'love': 42,
 'on': 43,
 'take': 44,
 'am': 45,
 'us': 46,
 'keep': 47,
 'need': 48,
 'home': 49,
 'let': 50,
 'stop': 51,
 'did': 52,
 'no': 53,
 'know': 54,
 'want': 55,
 'help': 56,
 'away': 57,
 'got': 58,
 'cant': 59,
 'hes': 60,
 'has': 61,
 'try': 62,
 'how': 63,
 'lost': 64,
 'may': 65,
 'now': 66,
 'one': 67,
 'look': 68,
 'there': 69,
 'out': 70,
 'saw': 71,
 'what': 72,
 'will': 73,
 'busy': 74,
 'lets': 75,
 'see': 76,
 'too': 77,
 'theyre': 78,
 'your': 79,
 'just': 80,
 'down': 81,
 'wait': 82,
 'for

In [56]:
def encode_sentences(tokenizer,length,sentences):
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X,length,padding='post')
    return X

In [57]:
def one_hot_encode_output(sentences,vocab_size):
    ylist = list()
    for X in sentences:
        y = to_categorical(X,num_classes=vocab_size)
        ylist.append(y)
    y_array = array(ylist)
    y_array = y_array.reshape(sentences.shape[0],sentences.shape[1],vocab_size)
    return y_array

In [58]:
# ENcoding English and German Sentences :
#trainX = encode_sentences(german_tokenizer,max_german_length,train_data[:,1])
trainX = encode_sentences(english_tokenizer,max_english_length,train_data[:,0])

In [59]:
trainX

array([[611, 544,   0,   0,   0],
       [ 63,   9,   3,   0,   0],
       [  2,  55,   6, 224,   0],
       ...,
       [  2, 152,  23, 569,   0],
       [ 75,  76,   3,   0,   0],
       [  1,   4, 521,   0,   0]])

In [60]:
trainX.shape

(9000, 5)

In [61]:
train_data[:,1]

array(['was kommt vor', 'wir war es', 'ich will arbeit', ...,
       'ich fuhle mein alter', 'zeigts mal her', 'tom ist erledigt'],
      dtype='<U69')

In [62]:
german_tokenizer.index_word[8]

'wir'

In [63]:
trainY = encode_sentences(german_tokenizer,max_german_length,train_data[:,1])
testX = encode_sentences(english_tokenizer,max_english_length,test_data[:,0])
testY = encode_sentences(german_tokenizer,max_german_length,test_data[:,1])

In [64]:
testX.shape

(1000, 5)

In [65]:
#trainY = one_hot_encode_output(trainY,english_vocab_size)
trainY = one_hot_encode_output(trainY,german_vocab_size)

In [66]:
#testY =  one_hot_encode_output(testY,english_vocab_size)
testY =  one_hot_encode_output(testY,german_vocab_size)

In [48]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model

# define model
model = define_model(english_vocab_size,german_vocab_size, max_english_length, max_german_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model_try1.png', show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 256)            566784    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 9, 256)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 9, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 9, 3526)           906182    
Total params: 2,523,590
Trainable params: 2,523,590
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
# fit model
filename = 'C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\model_eng_to_german.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=40, batch_size=64, validation_data=(testX, testY), callbacks=checkpoint, verbose=2)
#model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), verbose=2)

Epoch 1/40
141/141 - 29s - loss: 3.2307 - val_loss: 2.3785
Epoch 2/40
141/141 - 24s - loss: 2.2202 - val_loss: 2.2302
Epoch 3/40
141/141 - 24s - loss: 2.1155 - val_loss: 2.1752
Epoch 4/40
141/141 - 24s - loss: 2.0375 - val_loss: 2.1159
Epoch 5/40
141/141 - 24s - loss: 1.9452 - val_loss: 2.0350
Epoch 6/40
141/141 - 24s - loss: 1.8546 - val_loss: 1.9862
Epoch 7/40
141/141 - 24s - loss: 1.7772 - val_loss: 1.9353
Epoch 8/40
141/141 - 24s - loss: 1.6958 - val_loss: 1.8749
Epoch 9/40
141/141 - 24s - loss: 1.6114 - val_loss: 1.8165
Epoch 10/40
141/141 - 25s - loss: 1.5292 - val_loss: 1.7604
Epoch 11/40
141/141 - 25s - loss: 1.4571 - val_loss: 1.7238
Epoch 12/40
141/141 - 25s - loss: 1.3893 - val_loss: 1.6882
Epoch 13/40
141/141 - 25s - loss: 1.3248 - val_loss: 1.6541
Epoch 14/40
141/141 - 25s - loss: 1.2643 - val_loss: 1.6220
Epoch 15/40
141/141 - 25s - loss: 1.2028 - val_loss: 1.5904
Epoch 16/40
141/141 - 24s - loss: 1.1466 - val_loss: 1.5655
Epoch 17/40
141/141 - 25s - loss: 1.0911 - val_lo

In [69]:
from tensorflow.keras.models import load_model

In [97]:
# load model
filename = 'C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\model_eng_to_german_with_glove.h5'
model1 = load_model(filename)


In [71]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [72]:
# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

In [77]:
testX.shape

(1000, 5)

In [83]:
testX

array([[  7,  35, 359,   0,   0],
       [ 12,  61, 186,   0,   0],
       [229,  36,   0,   0,   0],
       ...,
       [ 89, 792,   0,   0,   0],
       [ 31,   4,  27,   0,   0],
       [ 12, 464,  95,   0,   0]])

In [85]:
testX[0].shape

(5,)

In [94]:
for i, source in enumerate(testX):
    # translate encoded source text
    print(source)
    print(source.shape)
    
    source = source.reshape((1, source.shape[0]))
    translation = predict_sequence(model1, german_tokenizer, source)
    print(translation)
    if(i==10):
        break

[  7  35 359   0   0]
(5,)
ich bin nicht
[ 12  61 186   0   0]
(5,)
er hat in in alt
[229  36   0   0   0]
(5,)
geschieden auf
[  1 467 155   0   0]
(5,)
tom kann nicht dein
[ 30 252   0   0   0]
(5,)
sie werden an
[76  5  0  0  0]
(5,)
summst uns
[   1 1125    0    0    0]
(5,)
tom winter sich
[ 2 37 16 89  0]
(5,)
ich deine singen
[   7 1428    0    0    0]
(5,)
ich bin
[ 227 1426    0    0    0]
(5,)
holen mal hause
[  28 1402    0    0    0]
(5,)
tom ist


In [92]:
one_sentence = ['they let me go']
source = array(one_sentence)
print(source)
print(source.shape)


['they let me go']
(1,)


In [93]:
source = encode_sentences(english_tokenizer,max_english_length,source)
print(source)
print(source.shape)

[[30 50  8 13  0]]
(1, 5)


In [87]:
type(source)

numpy.ndarray

In [106]:
english_tokenizer = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\english_tokenizer.pkl','rb'))
english_tokenizer.index_word

{1: 'tom',
 2: 'i',
 3: 'it',
 4: 'is',
 5: 'you',
 6: 'a',
 7: 'im',
 8: 'me',
 9: 'was',
 10: 'its',
 11: 'we',
 12: 'he',
 13: 'go',
 14: 'do',
 15: 'this',
 16: 'to',
 17: 'youre',
 18: 'are',
 19: 'that',
 20: 'were',
 21: 'ill',
 22: 'can',
 23: 'my',
 24: 'dont',
 25: 'get',
 26: 'come',
 27: 'she',
 28: 'toms',
 29: 'the',
 30: 'they',
 31: 'who',
 32: 'be',
 33: 'here',
 34: 'have',
 35: 'in',
 36: 'up',
 37: 'like',
 38: 'well',
 39: 'not',
 40: 'him',
 41: 'thats',
 42: 'love',
 43: 'on',
 44: 'take',
 45: 'am',
 46: 'us',
 47: 'keep',
 48: 'need',
 49: 'home',
 50: 'let',
 51: 'stop',
 52: 'did',
 53: 'no',
 54: 'know',
 55: 'want',
 56: 'help',
 57: 'away',
 58: 'got',
 59: 'cant',
 60: 'hes',
 61: 'has',
 62: 'try',
 63: 'how',
 64: 'lost',
 65: 'may',
 66: 'now',
 67: 'one',
 68: 'look',
 69: 'there',
 70: 'out',
 71: 'saw',
 72: 'what',
 73: 'will',
 74: 'busy',
 75: 'lets',
 76: 'see',
 77: 'too',
 78: 'theyre',
 79: 'your',
 80: 'just',
 81: 'down',
 82: 'wait',
 83: 

In [108]:
english_tokenizer = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\english_tokenizer.pkl','rb'))
german_tokenizer = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\german_tokenizer.pkl','rb'))

english_tokenizer.index_word.get(292)

'hello'

In [153]:
# Trial :
english_tokenizer = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\english_tokenizer.pkl','rb'))
german_tokenizer = load(open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\german_tokenizer.pkl','rb'))
filename = 'C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\model_eng_to_german_with_glove.h5'
model = load_model(filename)

filename = 'C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\model_eng_to_german.h5'
model1 = load_model(filename)

max_german_length = 9
max_english_length = 5
english_vocab_size = 2214
german_vocab_size = 3526



In [154]:
#sie haben mich gehen lassen

In [165]:
one_sentence = ['i live here']
source = array(one_sentence)
source = encode_sentences(english_tokenizer,max_english_length,source)
print(source)
#output = source
source = source.reshape((1, source.shape[1]))
translation = predict_sequence(model, german_tokenizer, source)
print(type(translation))
print(translation)

[[  2 354  33   0   0]]
<class 'str'>
ich hier hier hier


In [ ]:
['tom cooks well', 'keep it up', 'come here tom', ...,
       'i live here', 'take your time', 'are you done'

In [54]:
test_data[1]

array(['youre like me', 'du bist wie ich',
       'ccby france attribution tatoebaorg ck felixjp'], dtype='<U69')

In [55]:
test_data[1][0]

'youre like me'

In [56]:
testX[0]

array([398,  18, 476,   0,   0])

In [57]:
test_data[0]

array(['dogs are smart', 'hunde sind klug',
       'ccby france attribution tatoebaorg ck manfredo'], dtype='<U69')

In [134]:
german_tokenizer.index_word[6]

'das'

In [135]:
testX[0].shape

(9,)

In [136]:
t1 = testX[0].reshape(1,testX[0].shape[0])
t1

array([[ 6, 37, 34,  0,  0,  0,  0,  0,  0]])

In [62]:
test_data[0][0]

'dogs are smart'

In [63]:
# evaluate of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, german_tokenizer, source)
        raw_target, raw_src = raw_dataset[i][1],raw_dataset[i][0]
        if i < 20:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()])
        predicted.append(translation.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [64]:
from numpy import argmax
from nltk.translate.bleu_score import corpus_bleu


In [65]:
print('test')
evaluate_model(model1, german_tokenizer, testX, test_data)

test
src=[dogs are smart], target=[hunde sind klug], predicted=[hunde sind clever]
src=[youre like me], target=[du bist wie ich], predicted=[du hast mich]
src=[tom likes tea], target=[tom liebt tee], predicted=[tom mag tee]
src=[mine is black], target=[meine ist schwarz], predicted=[meiner ist schwarz]
src=[come aboard], target=[kommen sie an bord], predicted=[komm an bord]
src=[im bored too], target=[ich langweile mich auch], predicted=[mir ist auch auch]
src=[cows give milk], target=[kuhe geben milch], predicted=[sei sie kurz]
src=[lets find out], target=[finden wir es heraus], predicted=[lass wir den]
src=[ignore it], target=[beachte es gar nicht], predicted=[ignorier sie es]
src=[i wont come], target=[ich werde nicht kommen], predicted=[ich werde nicht kommen]
src=[ill get it], target=[ich werde es holen], predicted=[ich werde es]
src=[we need a map], target=[wir brauchen eine karte], predicted=[wir brauchen eine landkarte]
src=[hand it over], target=[handige es aus], predicted=[gi

# Embedding Layer using Glove Vectors !!!

from tqdm import tqdm
import numpy as np

In [71]:
embeddings_index = {}
f = open('C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\1835_3176_compressed_glove.6B.200d\\glove.6B.200d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400000it [01:13, 5417.49it/s]


Found 400000 word vectors.


In [72]:
english_tokenizer.word_index

{'tom': 1,
 'i': 2,
 'it': 3,
 'is': 4,
 'you': 5,
 'a': 6,
 'im': 7,
 'me': 8,
 'was': 9,
 'its': 10,
 'we': 11,
 'he': 12,
 'go': 13,
 'do': 14,
 'this': 15,
 'to': 16,
 'youre': 17,
 'are': 18,
 'that': 19,
 'were': 20,
 'ill': 21,
 'can': 22,
 'my': 23,
 'dont': 24,
 'get': 25,
 'come': 26,
 'she': 27,
 'toms': 28,
 'the': 29,
 'they': 30,
 'who': 31,
 'be': 32,
 'here': 33,
 'have': 34,
 'in': 35,
 'up': 36,
 'like': 37,
 'well': 38,
 'not': 39,
 'him': 40,
 'thats': 41,
 'love': 42,
 'on': 43,
 'take': 44,
 'am': 45,
 'us': 46,
 'keep': 47,
 'need': 48,
 'home': 49,
 'let': 50,
 'stop': 51,
 'did': 52,
 'no': 53,
 'know': 54,
 'want': 55,
 'help': 56,
 'away': 57,
 'got': 58,
 'hes': 59,
 'has': 60,
 'cant': 61,
 'try': 62,
 'lost': 63,
 'how': 64,
 'may': 65,
 'now': 66,
 'there': 67,
 'one': 68,
 'look': 69,
 'out': 70,
 'saw': 71,
 'will': 72,
 'what': 73,
 'busy': 74,
 'lets': 75,
 'see': 76,
 'too': 77,
 'theyre': 78,
 'your': 79,
 'down': 80,
 'just': 81,
 'wait': 82,
 'for

In [73]:
embeddings_index['the']

array([-7.1549e-02,  9.3459e-02,  2.3738e-02, -9.0339e-02,  5.6123e-02,
        3.2547e-01, -3.9796e-01, -9.2139e-02,  6.1181e-02, -1.8950e-01,
        1.3061e-01,  1.4349e-01,  1.1479e-02,  3.8158e-01,  5.4030e-01,
       -1.4088e-01,  2.4315e-01,  2.3036e-01, -5.5339e-01,  4.8154e-02,
        4.5662e-01,  3.2338e+00,  2.0199e-02,  4.9019e-02, -1.4132e-02,
        7.6017e-02, -1.1527e-01,  2.0060e-01, -7.7657e-02,  2.4328e-01,
        1.6368e-01, -3.4118e-01, -6.6070e-02,  1.0152e-01,  3.8232e-02,
       -1.7668e-01, -8.8153e-01, -3.3895e-01, -3.5481e-02, -5.5095e-01,
       -1.6899e-02, -4.3982e-01,  3.9004e-02,  4.0447e-01, -2.5880e-01,
        6.4594e-01,  2.6641e-01,  2.8009e-01, -2.4625e-02,  6.3302e-01,
       -3.1700e-01,  1.0271e-01,  3.0886e-01,  9.7792e-02, -3.8227e-01,
        8.6552e-02,  4.7075e-02,  2.3511e-01, -3.2127e-01, -2.8538e-01,
        1.6670e-01, -4.9707e-03, -6.2714e-01, -2.4904e-01,  2.9713e-01,
        1.4379e-01, -1.2325e-01, -5.8178e-02, -1.0290e-03, -8.21

In [74]:
len(embeddings_index['the'])

200

In [75]:
embedding_matrix = np.zeros((len(english_tokenizer.word_index) + 1, 200))
for word, i in tqdm(english_tokenizer.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████████████████████████████████████████████████████████████████████| 2213/2213 [00:00<00:00, 189540.64it/s]


In [1]:
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, 200, weights=[embedding_matrix],input_length=src_timesteps, trainable=False))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model

# define model
model = define_model(english_vocab_size,german_vocab_size, max_english_length, max_german_length, 200)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
#plot_model(model, to_file='model_try1.png', show_shapes=True)

NameError: name 'english_vocab_size' is not defined

In [86]:
# fit model
filename = 'C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\model_eng_to_german_with_glove.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=30, batch_size=32, validation_data=(testX, testY), callbacks=checkpoint, verbose=2)
#model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), verbose=2)

Epoch 1/30
282/282 - 23s - loss: 2.7094 - val_loss: 2.3455
Epoch 2/30
282/282 - 21s - loss: 2.2459 - val_loss: 2.2758
Epoch 3/30
282/282 - 21s - loss: 2.1673 - val_loss: 2.2131
Epoch 4/30
282/282 - 21s - loss: 2.0699 - val_loss: 2.0961
Epoch 5/30
282/282 - 21s - loss: 1.9141 - val_loss: 1.9615
Epoch 6/30
282/282 - 21s - loss: 1.7795 - val_loss: 1.8648
Epoch 7/30
282/282 - 21s - loss: 1.6752 - val_loss: 1.8032
Epoch 8/30
282/282 - 21s - loss: 1.5814 - val_loss: 1.7331
Epoch 9/30
282/282 - 21s - loss: 1.4898 - val_loss: 1.6667
Epoch 10/30
282/282 - 21s - loss: 1.4032 - val_loss: 1.6203
Epoch 11/30
282/282 - 21s - loss: 1.3177 - val_loss: 1.5739
Epoch 12/30
282/282 - 21s - loss: 1.2367 - val_loss: 1.5350
Epoch 13/30
282/282 - 21s - loss: 1.1609 - val_loss: 1.5096
Epoch 14/30
282/282 - 21s - loss: 1.0906 - val_loss: 1.4748
Epoch 15/30
282/282 - 21s - loss: 1.0234 - val_loss: 1.4495
Epoch 16/30
282/282 - 21s - loss: 0.9589 - val_loss: 1.4299
Epoch 17/30
282/282 - 21s - loss: 0.8983 - val_lo

In [87]:
print('test')
evaluate_model(model, german_tokenizer, testX, test_data)

test
src=[dogs are smart], target=[hunde sind klug], predicted=[hunde sind clever]
src=[youre like me], target=[du bist wie ich], predicted=[du hast dich euch]
src=[tom likes tea], target=[tom liebt tee], predicted=[tom mag nusse]
src=[mine is black], target=[meine ist schwarz], predicted=[meiner ist schwarz]
src=[come aboard], target=[kommen sie an bord], predicted=[komm an bord]
src=[im bored too], target=[ich langweile mich auch], predicted=[mir ist zu mude]
src=[cows give milk], target=[kuhe geben milch], predicted=[vogel madchen tee]
src=[lets find out], target=[finden wir es heraus], predicted=[lass uns zeit]
src=[ignore it], target=[beachte es gar nicht], predicted=[beachten es es nicht]
src=[i wont come], target=[ich werde nicht kommen], predicted=[ich werde nicht bereit]
src=[ill get it], target=[ich werde es holen], predicted=[ich werde es bekommen]
src=[we need a map], target=[wir brauchen eine karte], predicted=[wir brauchen eine landkarte]
src=[hand it over], target=[handi

In [78]:
print('test')
evaluate_model(model, german_tokenizer, testX, test_data)

test
src=[dogs are smart], target=[hunde sind klug], predicted=[hunde sind clever]
src=[youre like me], target=[du bist wie ich], predicted=[du bist mein chef]
src=[tom likes tea], target=[tom liebt tee], predicted=[tom mag essen]
src=[mine is black], target=[meine ist schwarz], predicted=[meiner ist schwarz]
src=[come aboard], target=[kommen sie an bord], predicted=[kommt an bord]
src=[im bored too], target=[ich langweile mich auch], predicted=[ich bin auch langweilig]
src=[cows give milk], target=[kuhe geben milch], predicted=[maria ist]
src=[lets find out], target=[finden wir es heraus], predicted=[lass uns uns]
src=[ignore it], target=[beachte es gar nicht], predicted=[beachten es es]
src=[i wont come], target=[ich werde nicht kommen], predicted=[ich werde nicht gehen]
src=[ill get it], target=[ich werde es holen], predicted=[ich werde es]
src=[we need a map], target=[wir brauchen eine karte], predicted=[wir brauchen eine landkarte]
src=[hand it over], target=[handige es aus], pred

In [4]:
# load model
filename = 'C:\\Users\\kvoth\\Desktop\\Datasets\\deu-eng\\model_eng_to_german_with_glove.h5'
model2 = load_model(filename)


In [ ]:
print('test')
evaluate_model(model2, german_tokenizer, testX, test_data)